In [5]:
%run datasetGenerator.py --ID 1002104402

Exception: File `'datasetGenerator.py'` not found.

In [ ]:
import csv
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact
odd_ratios=[]
p_values=[]
with open("1002104402.csv", 'r') as file:
    reader = csv.reader(file,delimiter='\t')
    header=next(reader)
    for row in reader:
        x = row[0].split(",")
        y=np.resize(np.array([list(map(float, x[1:]))]),(2,2))
        oddsr, p = fisher_exact(y, alternative='greater')
        odd_ratios.append(oddsr)
        p_values.append(p)
df = pd.read_csv("1002104402.csv")
df["odd_ratios"] = odd_ratios
df["p_values"]=p_values
df.to_csv("1002104402.csv", index=False)

In [ ]:
df

In [ ]:
result = pd.DataFrame(df[['SNP', 'p_values']])
result

In [ ]:
result = result.assign(significant= df['p_values']<5e-8)
result

In [ ]:
result['significant'].value_counts() [True]

In [ ]:
result

In [ ]:
result.to_csv("res.csv", index=False)

In [ ]:
result = result.assign(significant_after_corrected= df['p_values']<5e-11)
result

In [ ]:
result.to_csv("significant_after_corrected.csv", index=False)

In [ ]:
result['significant_after_corrected'].value_counts() [True]

In [ ]:
from pandas import DataFrame
from scipy.stats import uniform
from scipy.stats import randint
import numpy as np
import matplotlib.pyplot as plt

# -log_10(pvalue)
df['minuslog10pvalue'] = -np.log10(df.p_values)

# How to plot SNP vs. -log10(pvalue)?
df['SNP'] = range(len(df))
df_grouped = df.groupby(('SNP'))

fig = plt.figure()
ax = fig.add_subplot(111)
for num, (name, group) in enumerate(df_grouped):
    group.plot(kind='scatter', x='SNP', y='minuslog10pvalue', ax=ax, color='black')
    
#original threshold
plt.axhline(y = 8, color = 'b')
#corrected threshold
plt.axhline(y = 11, color = 'r')
ax.set_xlim([0, len(df.p_values)])
ax.set_ylim([0, 40])
ax.set_xlabel('SNP')